In [2]:
# IEOR4004 Project 2
import pandas as pd

games_df = pd.read_csv('games.csv')

teams = pd.unique(games_df[["Visitor", "Home"]].values.ravel())

summary_dict = {}
for team in teams:
    team_summary = {}

    home_dates = games_df.loc[games_df["Home"] == team, "Date"].tolist()
    team_summary['Home dates'] = home_dates

    away_dates = games_df.loc[games_df["Visitor"] == team, "Date"].tolist()
    team_summary['Away dates'] = away_dates

    home_counts = (
        games_df.loc[games_df["Home"] == team, "Visitor"]
        .value_counts()
        .rename_axis("Opponent")
        .reset_index(name="Home Games")
    )
    away_counts = (
        games_df.loc[games_df["Visitor"] == team, "Home"]
        .value_counts()
        .rename_axis("Opponent")
        .reset_index(name="Away Games")
    )
    total_counts = pd.merge(home_counts,
                            away_counts,
                            on="Opponent",
                            how="outer").fillna(0)
    team_summary['H/A Games vs. Others'] = total_counts

    summary_dict[team] = team_summary
    print(f"\n {team} Summary:")
    print(f"Home dates: {team_summary['Home dates']}")
    print(f"Away dates: {team_summary['Away dates']}")
    print("Head-to-head counts:")
    print(team_summary['H/A Games vs. Others'])


 Golden State Warriors Summary:
Home dates: ['Mon, Nov 03, 2025', 'Wed, Nov 05, 2025', 'Tue, Nov 11, 2025', 'Sat, Nov 15, 2025', 'Mon, Nov 17, 2025', 'Thu, Nov 27, 2025', 'Thu, Dec 25, 2025']
Away dates: ['Sat, Nov 01, 2025', 'Fri, Nov 07, 2025', 'Thu, Nov 13, 2025', 'Wed, Nov 19, 2025', 'Fri, Nov 21, 2025', 'Sun, Nov 23, 2025', 'Fri, Nov 28, 2025', 'Sat, Nov 29, 2025', 'Mon, Dec 01, 2025']
Head-to-head counts:
               Opponent  Home Games  Away Games
0         Atlanta Hawks         0.0         1.0
1        Boston Celtics         1.0         1.0
2         Brooklyn Nets         0.0         1.0
3         Chicago Bulls         0.0         1.0
4   Cleveland Cavaliers         1.0         0.0
5      Dallas Mavericks         0.0         1.0
6        Denver Nuggets         1.0         0.0
7       Houston Rockets         1.0         0.0
8    Los Angeles Lakers         0.0         1.0
9            Miami Heat         0.0         1.0
10      Milwaukee Bucks         1.0         0.0
11      

In [3]:
# Export team summaries to Excel
import pandas as pd

# Create Excel writer object
with pd.ExcelWriter('team_schedules_summary.xlsx', engine='openpyxl') as writer:
    
    # Create a summary sheet with all teams
    summary_data = []
    for team in teams:
        summary_data.append({
            'Team': team,
            'Total Home Games': len(summary_dict[team]['Home dates']),
            'Total Away Games': len(summary_dict[team]['Away dates']),
            'Total Games': len(summary_dict[team]['Home dates']) + len(summary_dict[team]['Away dates'])
        })
    
    summary_df = pd.DataFrame(summary_data)
    summary_df.to_excel(writer, sheet_name='Overall Summary', index=False)
    
    # Create a sheet for each team with detailed information
    for team in teams:
        # Clean team name for sheet name (Excel has 31 char limit and special char restrictions)
        sheet_name = team.replace(' ', '_')[:31]
        
        # Create a DataFrame with home and away dates
        max_len = max(len(summary_dict[team]['Home dates']), 
                     len(summary_dict[team]['Away dates']))
        
        dates_df = pd.DataFrame({
            'Home Dates': summary_dict[team]['Home dates'] + [''] * (max_len - len(summary_dict[team]['Home dates'])),
            'Away Dates': summary_dict[team]['Away dates'] + [''] * (max_len - len(summary_dict[team]['Away dates']))
        })
        
        # Write dates to the sheet
        dates_df.to_excel(writer, sheet_name=sheet_name, index=False, startrow=0)
        
        # Write opponent matchup counts below the dates
        matchup_df = summary_dict[team]['H/A Games vs. Others']
        matchup_df.to_excel(writer, sheet_name=sheet_name, 
                           index=False, startrow=max_len + 3)
        
        # Add a header for the matchup section
        worksheet = writer.sheets[sheet_name]
        worksheet.cell(row=max_len + 3, column=1, value='Opponent Matchups:')

print("Excel file 'team_schedules_summary.xlsx' created successfully!")
print(f"\nThe file contains:")
print(f"  - 'Overall Summary' sheet with game counts for all teams")
print(f"  - {len(teams)} individual team sheets with detailed schedules and matchups")
print(f"\nTotal teams: {len(teams)}")
print(f"Total sheets: {len(teams) + 1}")

Excel file 'team_schedules_summary.xlsx' created successfully!

The file contains:
  - 'Overall Summary' sheet with game counts for all teams
  - 16 individual team sheets with detailed schedules and matchups

Total teams: 16
Total sheets: 17


In [4]:
# Export to CSV files for easy reading and analysis

# 1. Create a comprehensive game schedule CSV (long format - easy to filter and analyze)
all_games = []
for team in teams:
    # Home games
    for date in summary_dict[team]['Home dates']:
        opponents = games_df[(games_df['Home'] == team) & (games_df['Date'] == date)]['Visitor'].tolist()
        for opponent in opponents:
            all_games.append({
                'Date': date,
                'Team': team,
                'Opponent': opponent,
                'Location': 'Home',
                'Arena': games_df[(games_df['Home'] == team) & (games_df['Date'] == date)]['Arena'].iloc[0],
                'LOG': games_df[(games_df['Home'] == team) & (games_df['Date'] == date)]['LOG'].iloc[0]
            })
    
    # Away games
    for date in summary_dict[team]['Away dates']:
        opponents = games_df[(games_df['Visitor'] == team) & (games_df['Date'] == date)]['Home'].tolist()
        for opponent in opponents:
            all_games.append({
                'Date': date,
                'Team': team,
                'Opponent': opponent,
                'Location': 'Away',
                'Arena': games_df[(games_df['Visitor'] == team) & (games_df['Date'] == date)]['Arena'].iloc[0],
                'LOG': games_df[(games_df['Visitor'] == team) & (games_df['Date'] == date)]['LOG'].iloc[0]
            })

# Convert to DataFrame and save
games_schedule_df = pd.DataFrame(all_games)
games_schedule_df = games_schedule_df.sort_values(['Team', 'Date']).reset_index(drop=True)
games_schedule_df.to_csv('team_game_schedule.csv', index=False)

print(" Created 'team_game_schedule.csv'")
print(f"   Contains {len(games_schedule_df)} game records (team perspective)")
print(f"   Columns: {list(games_schedule_df.columns)}")


# 2. Create a team summary CSV
team_summary_data = []
for team in teams:
    team_summary_data.append({
        'Team': team,
        'Total_Games': len(summary_dict[team]['Home dates']) + len(summary_dict[team]['Away dates']),
        'Home_Games': len(summary_dict[team]['Home dates']),
        'Away_Games': len(summary_dict[team]['Away dates'])
    })

team_summary_df = pd.DataFrame(team_summary_data)
team_summary_df = team_summary_df.sort_values('Team').reset_index(drop=True)
team_summary_df.to_csv('team_summary.csv', index=False)

print("\n Created 'team_summary.csv'")
print(f"   Contains {len(team_summary_df)} teams")
print(f"   Columns: {list(team_summary_df.columns)}")


# 3. Create a matchup matrix CSV (opponent vs opponent)
matchup_data = []
for team in teams:
    matchup_df = summary_dict[team]['H/A Games vs. Others']
    for _, row in matchup_df.iterrows():
        matchup_data.append({
            'Team': team,
            'Opponent': row['Opponent'],
            'Home_Games': int(row['Home Games']),
            'Away_Games': int(row['Away Games']),
            'Total_Games': int(row['Home Games'] + row['Away Games'])
        })

matchup_matrix_df = pd.DataFrame(matchup_data)
matchup_matrix_df = matchup_matrix_df.sort_values(['Team', 'Opponent']).reset_index(drop=True)
matchup_matrix_df.to_csv('team_matchup_matrix.csv', index=False)

print("\n Created 'team_matchup_matrix.csv'")
print(f"   Contains {len(matchup_matrix_df)} team-opponent pairs")
print(f"   Columns: {list(matchup_matrix_df.columns)}")


# 4. Display previews
print("\n" + "="*70)
print("PREVIEW: Team Game Schedule (first 10 rows)")
print("="*70)
print(games_schedule_df.head(10).to_string(index=False))

print("\n" + "="*70)
print("PREVIEW: Team Summary")
print("="*70)
print(team_summary_df.to_string(index=False))

print("\n" + "="*70)
print("PREVIEW: Matchup Matrix (first 10 rows)")
print("="*70)
print(matchup_matrix_df.head(10).to_string(index=False))

print("\n" + "="*70)
print("FILES CREATED:")
print("="*70)
print("1. team_game_schedule.csv   - Complete game schedule for each team")
print("2. team_summary.csv          - Game count summary for all teams")
print("3. team_matchup_matrix.csv   - Head-to-head matchups between teams")
print("="*70)

 Created 'team_game_schedule.csv'
   Contains 256 game records (team perspective)
   Columns: ['Date', 'Team', 'Opponent', 'Location', 'Arena', 'LOG']

 Created 'team_summary.csv'
   Contains 16 teams
   Columns: ['Team', 'Total_Games', 'Home_Games', 'Away_Games']

 Created 'team_matchup_matrix.csv'
   Contains 240 team-opponent pairs
   Columns: ['Team', 'Opponent', 'Home_Games', 'Away_Games', 'Total_Games']

PREVIEW: Team Game Schedule (first 10 rows)
             Date          Team              Opponent Location                      Arena     LOG
Fri, Nov 07, 2025 Atlanta Hawks Golden State Warriors     Home           State Farm Arena 7:30 PM
Fri, Nov 21, 2025 Atlanta Hawks    Philadelphia 76ers     Away         Wells Fargo Center 7:30 PM
Fri, Nov 28, 2025 Atlanta Hawks          Phoenix Suns     Home           State Farm Arena 7:30 PM
Mon, Dec 01, 2025 Atlanta Hawks   Cleveland Cavaliers     Away Rocket Mortgage FieldHouse 7:30 PM
Mon, Nov 03, 2025 Atlanta Hawks       Toronto Raptor

In [5]:
# Create chronological schedule for each team with Home/Away indicator

# Create a list to store all team schedules
all_team_schedules = []

for team in teams:
    # Combine home and away games
    team_schedule = []
    
    # Add home games
    for date in summary_dict[team]['Home dates']:
        opponent = games_df[(games_df['Home'] == team) & (games_df['Date'] == date)]['Visitor'].iloc[0]
        arena = games_df[(games_df['Home'] == team) & (games_df['Date'] == date)]['Arena'].iloc[0]
        team_schedule.append({
            'Team': team,
            'Date': date,
            'Location': 'Home',
            'Opponent': opponent,
            'Arena': arena
        })
    
    # Add away games
    for date in summary_dict[team]['Away dates']:
        opponent = games_df[(games_df['Visitor'] == team) & (games_df['Date'] == date)]['Home'].iloc[0]
        arena = games_df[(games_df['Visitor'] == team) & (games_df['Date'] == date)]['Arena'].iloc[0]
        team_schedule.append({
            'Team': team,
            'Date': date,
            'Location': 'Away',
            'Opponent': opponent,
            'Arena': arena
        })
    
    all_team_schedules.extend(team_schedule)

# Create DataFrame
team_chronological_df = pd.DataFrame(all_team_schedules)

# Convert Date to datetime for proper sorting
team_chronological_df['Date'] = pd.to_datetime(team_chronological_df['Date'])

# Sort by Team and Date
team_chronological_df = team_chronological_df.sort_values(['Team', 'Date']).reset_index(drop=True)

# Add game number for each team
team_chronological_df['Game_Number'] = team_chronological_df.groupby('Team').cumcount() + 1

# Reorder columns
team_chronological_df = team_chronological_df[['Team', 'Game_Number', 'Date', 'Location', 'Opponent', 'Arena']]

# Save to CSV
team_chronological_df.to_csv('team_chronological_schedule.csv', index=False)

print(" Created 'team_chronological_schedule.csv'")
print(f"   Contains {len(team_chronological_df)} games")
print(f"   Columns: {list(team_chronological_df.columns)}")

# Display sample for each team
print("\n" + "="*80)
print("SAMPLE: First 5 games for each team (chronological order)")
print("="*80)

for team in sorted(teams):
    team_games = team_chronological_df[team_chronological_df['Team'] == team].head(5)
    print(f"\n{team}:")
    print(team_games.to_string(index=False))

# Overall statistics
print("\n" + "="*80)
print("STATISTICS:")
print("="*80)
for team in sorted(teams):
    team_data = team_chronological_df[team_chronological_df['Team'] == team]
    home_count = (team_data['Location'] == 'Home').sum()
    away_count = (team_data['Location'] == 'Away').sum()
    print(f"{team:30s} - Total: {len(team_data):2d} | Home: {home_count:2d} | Away: {away_count:2d}")

print("\n" + "="*80)
print("FILE SAVED: team_chronological_schedule.csv")
print("This file contains all teams' schedules in chronological order")
print("="*80)

 Created 'team_chronological_schedule.csv'
   Contains 256 games
   Columns: ['Team', 'Game_Number', 'Date', 'Location', 'Opponent', 'Arena']

SAMPLE: First 5 games for each team (chronological order)

Atlanta Hawks:
         Team  Game_Number       Date Location              Opponent            Arena
Atlanta Hawks            1 2025-11-01     Away         Chicago Bulls    United Center
Atlanta Hawks            2 2025-11-03     Home       Toronto Raptors State Farm Arena
Atlanta Hawks            3 2025-11-05     Away         Brooklyn Nets  Barclays Center
Atlanta Hawks            4 2025-11-07     Home Golden State Warriors State Farm Arena
Atlanta Hawks            5 2025-11-11     Away        Denver Nuggets       Ball Arena

Boston Celtics:
          Team  Game_Number       Date Location              Opponent            Arena
Boston Celtics            1 2025-11-01     Home Golden State Warriors        TD Garden
Boston Celtics            2 2025-11-03     Away    Los Angeles Lakers Crypto

**Question 3**

In [6]:
# Add timezone column to team_chronological_schedule.csv
# Eastern Time = 0, Central Time = -1, Mountain Time = -2, Pacific Time = -3

# Define timezone mapping for each arena
arena_timezone = {
    # Eastern Time (0)
    'TD Garden': 0,  # Boston
    'Barclays Center': 0,  # Brooklyn
    'Madison Square Garden': 0,  # New York
    'Wells Fargo Center': 0,  # Philadelphia
    'Scotiabank Arena': 0,  # Toronto
    'State Farm Arena': 0,  # Atlanta
    'Kaseya Center': 0,  # Miami
    'Rocket Mortgage FieldHouse': 0,  # Cleveland
    
    # Central Time (-1)
    'United Center': -1,  # Chicago
    'Fiserv Forum': -1,  # Milwaukee
    'American Airlines Center': -1,  # Dallas
    'Toyota Center': -1,  # Houston
    
    # Mountain Time (-2)
    'Ball Arena': -2,  # Denver
    'Footprint Center': -2,  # Phoenix
    
    # Pacific Time (-3)
    'Chase Center': -3,  # Golden State
    'Crypto.com Arena': -3  # Los Angeles Lakers
}

# Read the chronological schedule
team_chrono_df = pd.read_csv('team_chronological_schedule.csv')

# Add timezone column
team_chrono_df['Timezone'] = team_chrono_df['Arena'].map(arena_timezone)

# Reorder columns to place Timezone after Arena
team_chrono_df = team_chrono_df[['Team', 'Game_Number', 'Date', 'Location', 'Opponent', 'Arena', 'Timezone']]

# Save updated file
team_chrono_df.to_csv('team_chronological_schedule.csv', index=False)

print(" Updated 'team_chronological_schedule.csv' with Timezone column")
print(f"   Total games: {len(team_chrono_df)}")
print(f"   Columns: {list(team_chrono_df.columns)}")

# Display timezone distribution
print("\n" + "="*80)
print("TIMEZONE DISTRIBUTION:")
print("="*80)
timezone_counts = team_chrono_df['Timezone'].value_counts().sort_index()
timezone_names = {0: 'Eastern', -1: 'Central', -2: 'Mountain', -3: 'Pacific'}
for tz_value, count in timezone_counts.items():
    tz_name = timezone_names.get(tz_value, 'Unknown')
    print(f"{tz_name:12s} (Timezone = {tz_value:2d}): {count:3d} games")

# Display sample with timezone info
print("\n" + "="*80)
print("SAMPLE: First 10 games with timezone information")
print("="*80)
print(team_chrono_df.head(10).to_string(index=False))

# Verify all arenas have timezone assigned
missing_tz = team_chrono_df[team_chrono_df['Timezone'].isna()]
if len(missing_tz) > 0:
    print("\n  WARNING: Some arenas are missing timezone assignments:")
    print(missing_tz[['Arena']].drop_duplicates())
else:
    print("\n All arenas have timezone assignments")


 Updated 'team_chronological_schedule.csv' with Timezone column
   Total games: 256
   Columns: ['Team', 'Game_Number', 'Date', 'Location', 'Opponent', 'Arena', 'Timezone']

TIMEZONE DISTRIBUTION:
Pacific      (Timezone = -3):  28 games
Mountain     (Timezone = -2):  32 games
Central      (Timezone = -1):  72 games
Eastern      (Timezone =  0): 124 games

SAMPLE: First 10 games with timezone information
         Team  Game_Number       Date Location              Opponent              Arena  Timezone
Atlanta Hawks            1 2025-11-01     Away         Chicago Bulls      United Center        -1
Atlanta Hawks            2 2025-11-03     Home       Toronto Raptors   State Farm Arena         0
Atlanta Hawks            3 2025-11-05     Away         Brooklyn Nets    Barclays Center         0
Atlanta Hawks            4 2025-11-07     Home Golden State Warriors   State Farm Arena         0
Atlanta Hawks            5 2025-11-11     Away        Denver Nuggets         Ball Arena        -2
Atlan

In [7]:
# Check for infeasible triples: 3 consecutive games where sum of |timezone differences| >= 4

# Read the chronological schedule with timezones
team_chrono_df = pd.read_csv('team_chronological_schedule.csv')

# Store all infeasible triples
infeasible_triples = []

# Check each team's schedule
for team in sorted(team_chrono_df['Team'].unique()):
    team_games = team_chrono_df[team_chrono_df['Team'] == team].sort_values('Game_Number').reset_index(drop=True)
    
    # Check each consecutive triple of games
    for i in range(len(team_games) - 2):
        game1 = team_games.iloc[i]
        game2 = team_games.iloc[i + 1]
        game3 = team_games.iloc[i + 2]
        
        # Get timezones
        tz1 = game1['Timezone']
        tz2 = game2['Timezone']
        tz3 = game3['Timezone']
        
        # Calculate absolute differences between consecutive games
        diff_1_2 = abs(tz2 - tz1)
        diff_2_3 = abs(tz3 - tz2)
        total_diff = diff_1_2 + diff_2_3
        
        # Check if sum of absolute differences >= 4
        if total_diff >= 4:
            infeasible_triples.append({
                'Team': team,
                'Game1_Num': game1['Game_Number'],
                'Game1_Date': game1['Date'],
                'Game1_Location': game1['Location'],
                'Game1_Opponent': game1['Opponent'],
                'Game1_Arena': game1['Arena'],
                'Game1_TZ': tz1,
                'Game2_Num': game2['Game_Number'],
                'Game2_Date': game2['Date'],
                'Game2_Location': game2['Location'],
                'Game2_Opponent': game2['Opponent'],
                'Game2_Arena': game2['Arena'],
                'Game2_TZ': tz2,
                'Game3_Num': game3['Game_Number'],
                'Game3_Date': game3['Date'],
                'Game3_Location': game3['Location'],
                'Game3_Opponent': game3['Opponent'],
                'Game3_Arena': game3['Arena'],
                'Game3_TZ': tz3,
                'TZ_Diff_1_2': diff_1_2,
                'TZ_Diff_2_3': diff_2_3,
                'Total_TZ_Diff': total_diff
            })

# Convert to DataFrame
infeasible_df = pd.DataFrame(infeasible_triples)

# Save to CSV
if len(infeasible_df) > 0:
    infeasible_df.to_csv('infeasible_triples.csv', index=False)
    
    print("=" * 100)
    print("  INFEASIBLE TRIPLES FOUND")
    print("=" * 100)
    print(f"\nTotal infeasible triples: {len(infeasible_df)}")
    print(f"Teams affected: {infeasible_df['Team'].nunique()}")
    print(f"\nAffected teams: {sorted(infeasible_df['Team'].unique())}")
    
    # Summary by team
    print("\n" + "=" * 100)
    print("SUMMARY BY TEAM:")
    print("=" * 100)
    team_summary = infeasible_df.groupby('Team').size().reset_index(name='Count')
    print(team_summary.to_string(index=False))
    
    # Display all infeasible triples
    print("\n" + "=" * 100)
    print("DETAILED LIST OF ALL INFEASIBLE TRIPLES:")
    print("=" * 100)
    
    for idx, row in infeasible_df.iterrows():
        print(f"\n{row['Team']} - Triple #{idx + 1}")
        print(f"  Games {int(row['Game1_Num'])}-{int(row['Game2_Num'])}-{int(row['Game3_Num'])}")
        print(f"  Game {int(row['Game1_Num'])}: {row['Game1_Date']} - {row['Game1_Location']:4s} vs {row['Game1_Opponent']:25s} at {row['Game1_Arena']:35s} (TZ = {int(row['Game1_TZ']):2d})")
        print(f"  Game {int(row['Game2_Num'])}: {row['Game2_Date']} - {row['Game2_Location']:4s} vs {row['Game2_Opponent']:25s} at {row['Game2_Arena']:35s} (TZ = {int(row['Game2_TZ']):2d})")
        print(f"  Game {int(row['Game3_Num'])}: {row['Game3_Date']} - {row['Game3_Location']:4s} vs {row['Game3_Opponent']:25s} at {row['Game3_Arena']:35s} (TZ = {int(row['Game3_TZ']):2d})")
        print(f"  → TZ differences: |{int(row['Game2_TZ'])} - {int(row['Game1_TZ'])}| + |{int(row['Game3_TZ'])} - {int(row['Game2_TZ'])}| = {int(row['TZ_Diff_1_2'])} + {int(row['TZ_Diff_2_3'])} = {int(row['Total_TZ_Diff'])} ≥ 4 ⚠️")
    
    print("\n" + "=" * 100)
    print(f" Saved to 'infeasible_triples.csv'")
    print("=" * 100)
    
else:
    print("=" * 100)
    print(" NO INFEASIBLE TRIPLES FOUND")
    print("=" * 100)
    print("\nAll teams have feasible schedules!")
    print("No team has 3 consecutive games where the sum of timezone differences >= 4")
    print("=" * 100)


  INFEASIBLE TRIPLES FOUND

Total infeasible triples: 22
Teams affected: 12

Affected teams: ['Boston Celtics', 'Brooklyn Nets', 'Cleveland Cavaliers', 'Denver Nuggets', 'Golden State Warriors', 'Los Angeles Lakers', 'Miami Heat', 'Milwaukee Bucks', 'New York Knicks', 'Philadelphia 76ers', 'Phoenix Suns', 'Toronto Raptors']

SUMMARY BY TEAM:
                 Team  Count
       Boston Celtics      1
        Brooklyn Nets      1
  Cleveland Cavaliers      2
       Denver Nuggets      1
Golden State Warriors      4
   Los Angeles Lakers      3
           Miami Heat      1
      Milwaukee Bucks      1
      New York Knicks      2
   Philadelphia 76ers      1
         Phoenix Suns      3
      Toronto Raptors      2

DETAILED LIST OF ALL INFEASIBLE TRIPLES:

Boston Celtics - Triple #1
  Games 1-2-3
  Game 1: 2025-11-01 - Home vs Golden State Warriors     at TD Garden                           (TZ =  0)
  Game 2: 2025-11-03 - Away vs Los Angeles Lakers        at Crypto.com Arena             

## Optimization Model

In [8]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np

# Read original data
games_df = pd.read_csv('games.csv')
team_chrono_df = pd.read_csv('team_chronological_schedule.csv')

# Get team to timezone mapping (home arena timezone)
team_home_tz = {}
for team in team_chrono_df['Team'].unique():
    home_games = team_chrono_df[(team_chrono_df['Team'] == team) & (team_chrono_df['Location'] == 'Home')]
    if len(home_games) > 0:
        team_home_tz[team] = home_games.iloc[0]['Timezone']

print("Team Home Arena Timezones:")
for team, tz in sorted(team_home_tz.items()):
    tz_name = {0: 'Eastern', -1: 'Central', -2: 'Mountain', -3: 'Pacific'}[tz]
    print(f"  {team:30s}: {tz_name:10s} (TZ = {tz})")

# Create matchup list from original games (date, team1, team2)
matchups = []
for _, row in games_df.iterrows():
    matchups.append({
        'date': row['Date'],
        'team1': row['Visitor'],  # Originally visitor
        'team2': row['Home'],      # Originally home
        'original_home': row['Home'],
        'arena_if_team1_home': None,  # Will determine based on decision
        'arena_if_team2_home': None
    })

print(f"\nTotal matchups to schedule: {len(matchups)}")
print(f"Total teams: {len(team_home_tz)}")


Team Home Arena Timezones:
  Atlanta Hawks                 : Eastern    (TZ = 0)
  Boston Celtics                : Eastern    (TZ = 0)
  Brooklyn Nets                 : Eastern    (TZ = 0)
  Chicago Bulls                 : Central    (TZ = -1)
  Cleveland Cavaliers           : Eastern    (TZ = 0)
  Dallas Mavericks              : Central    (TZ = -1)
  Denver Nuggets                : Mountain   (TZ = -2)
  Golden State Warriors         : Pacific    (TZ = -3)
  Houston Rockets               : Central    (TZ = -1)
  Los Angeles Lakers            : Pacific    (TZ = -3)
  Miami Heat                    : Eastern    (TZ = 0)
  Milwaukee Bucks               : Central    (TZ = -1)
  New York Knicks               : Eastern    (TZ = 0)
  Philadelphia 76ers            : Eastern    (TZ = 0)
  Phoenix Suns                  : Mountain   (TZ = -2)
  Toronto Raptors               : Eastern    (TZ = 0)

Total matchups to schedule: 128
Total teams: 16


In [9]:
# Build Gurobi Model
model = gp.Model("NBA_Schedule_Optimization")

# Decision Variables: x[m] = 1 if team2 is home in matchup m, 0 if team1 is home
x = model.addVars(len(matchups), vtype=GRB.BINARY, name="home_assignment")

# Auxiliary variables: timezone for each team's game
# For each team t and their game g, know which timezone they'll be in
teams_list = sorted(team_home_tz.keys())
teams = {team: [] for team in teams_list}  # List of (matchup_idx, is_team2) for each team

# Build team game assignments
for m, matchup in enumerate(matchups):
    team1, team2 = matchup['team1'], matchup['team2']
    teams[team1].append((m, False))  # team1 is first in matchup
    teams[team2].append((m, True))   # team2 is second in matchup

# Sort each team's games by date to get chronological order
for team in teams_list:
    teams[team] = sorted(teams[team], key=lambda x: matchups[x[0]]['date'])

print(f"\nGames per team:")
for team in teams_list:
    print(f"  {team:30s}: {len(teams[team])} games")

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter WLSSecret
Set parameter LicenseID to value 2705371
Academic license 2705371 - for non-commercial use only - registered to rc___@columbia.edu

Games per team:
  Atlanta Hawks                 : 16 games
  Boston Celtics                : 16 games
  Brooklyn Nets                 : 16 games
  Chicago Bulls                 : 16 games
  Cleveland Cavaliers           : 16 games
  Dallas Mavericks              : 16 games
  Denver Nuggets                : 16 games
  Golden State Warriors         : 16 games
  Houston Rockets               : 16 games
  Los Angeles Lakers            : 16 games
  Miami Heat                    : 16 games
  Milwaukee Bucks               : 16 games
  New York Knicks               : 16 games
  Philadelphia 76ers            : 16 games
  Phoenix Suns                  : 16 games
  Toronto Raptors               : 16 games
Set parameter LicenseID to value 2705371
Academic license 2705371 - for non-commercial use

In [13]:
# Add constraints

# 1. Balance constraint: Each team should have equal home and away games
# Original distribution
original_home_counts = {}
for team in teams_list:
    home_count = sum(1 for m, is_team2 in teams[team] 
                     if matchups[m]['original_home'] == team)
    original_home_counts[team] = home_count

print("Original home game counts:")
for team, count in sorted(original_home_counts.items()):
    total = len(teams[team])
    print(f"  {team:30s}: {count}/{total} home games")

# Constraint: Each team must have the same number of home games as original
for team in teams_list:
    team_home_games = []
    for m, is_team2 in teams[team]:
        if is_team2:
            # This team is team2 in matchup m, home if x[m] = 1
            team_home_games.append(x[m])
        else:
            # This team is team1 in matchup m, home if x[m] = 0
            team_home_games.append(1 - x[m])
    
    model.addConstr(
        gp.quicksum(team_home_games) == original_home_counts[team],
        name=f"home_balance_{team}"
    )

print(f"\n Added {len(teams_list)} home/away balance constraints")


Original home game counts:
  Atlanta Hawks                 : 10/16 home games
  Boston Celtics                : 6/16 home games
  Brooklyn Nets                 : 7/16 home games
  Chicago Bulls                 : 9/16 home games
  Cleveland Cavaliers           : 10/16 home games
  Dallas Mavericks              : 9/16 home games
  Denver Nuggets                : 8/16 home games
  Golden State Warriors         : 7/16 home games
  Houston Rockets               : 9/16 home games
  Los Angeles Lakers            : 7/16 home games
  Miami Heat                    : 8/16 home games
  Milwaukee Bucks               : 9/16 home games
  New York Knicks               : 6/16 home games
  Philadelphia 76ers            : 7/16 home games
  Phoenix Suns                  : 8/16 home games
  Toronto Raptors               : 8/16 home games

 Added 16 home/away balance constraints


In [10]:
# 2. Infeasible triple constraints
# For each team and each consecutive triple of games, ensure sum of TZ differences < 4

# tz[team][g] = timezone where team plays game g
tz = {}
for team in teams_list:
    tz[team] = {}
    for g in range(len(teams[team])):
        m, is_team2 = teams[team][g]
        team1, team2 = matchups[m]['team1'], matchups[m]['team2']
        
        # Timezone depends on who is home
        # If team is home: timezone = team's home timezone
        # If team is away: timezone = opponent's home timezone
        
        if is_team2:
            # This team is team2
            # If x[m] = 1: team2 is home, tz = team_home_tz[team]
            # If x[m] = 0: team1 is home, tz = team_home_tz[team1]
            tz_if_home = team_home_tz[team]
            tz_if_away = team_home_tz[team1]
        else:
            # This team is team1
            # If x[m] = 0: team1 is home, tz = team_home_tz[team]
            # If x[m] = 1: team2 is home, tz = team_home_tz[team2]
            tz_if_home = team_home_tz[team]
            tz_if_away = team_home_tz[team2]
        
        # Create continuous variable for timezone
        tz[team][g] = model.addVar(lb=-3, ub=0, vtype=GRB.INTEGER, name=f"tz_{team}_{g}")
        
        # Linearize: tz[team][g] = tz_if_home * (1 - x[m]) + tz_if_away * x[m] (if team is team1)
        # or: tz[team][g] = tz_if_home * x[m] + tz_if_away * (1 - x[m]) (if team is team2)
        if is_team2:
            # tz = tz_if_home * x[m] + tz_if_away * (1 - x[m])
            model.addConstr(
                tz[team][g] == tz_if_home * x[m] + tz_if_away * (1 - x[m]),
                name=f"tz_def_{team}_{g}"
            )
        else:
            # tz = tz_if_home * (1 - x[m]) + tz_if_away * x[m]
            model.addConstr(
                tz[team][g] == tz_if_home * (1 - x[m]) + tz_if_away * x[m],
                name=f"tz_def_{team}_{g}"
            )

print(f" Created timezone variables for all team games")


 Created timezone variables for all team games


In [11]:
# Add absolute value constraints and triple feasibility constraints
# For each team's consecutive triple of games (g, g+1, g+2):
# |tz[g+1] - tz[g]| + |tz[g+2] - tz[g+1]| < 4

triple_constraints_count = 0

for team in teams_list:
    num_games = len(teams[team])
    
    # For each consecutive triple
    for g in range(num_games - 2):
        # Create auxiliary variables for absolute values
        diff1 = model.addVar(lb=0, ub=3, vtype=GRB.INTEGER, name=f"diff1_{team}_{g}")
        diff2 = model.addVar(lb=0, ub=3, vtype=GRB.INTEGER, name=f"diff2_{team}_{g}")
        
        # Linearize |tz[g+1] - tz[g]|
        model.addConstr(diff1 >= tz[team][g+1] - tz[team][g], name=f"abs1a_{team}_{g}")
        model.addConstr(diff1 >= tz[team][g] - tz[team][g+1], name=f"abs1b_{team}_{g}")
        
        # Linearize |tz[g+2] - tz[g+1]|
        model.addConstr(diff2 >= tz[team][g+2] - tz[team][g+1], name=f"abs2a_{team}_{g}")
        model.addConstr(diff2 >= tz[team][g+1] - tz[team][g+2], name=f"abs2b_{team}_{g}")
        
        # Add constraint: diff1 + diff2 <= 3 (strictly less than 4)
        model.addConstr(
            diff1 + diff2 <= 3,
            name=f"triple_feasible_{team}_{g}"
        )
        
        triple_constraints_count += 1

print(f" Added {triple_constraints_count} triple feasibility constraints")
print(f"   (Ensures no team has 3 consecutive games with sum of TZ differences >= 4)")


 Added 224 triple feasibility constraints
   (Ensures no team has 3 consecutive games with sum of TZ differences >= 4)


In [12]:
# Objective: Minimize changes from original schedule
# Minimize the number of matchups where we flip home/away from original

# x[m] = 1 means team2 is home
# Original: team2 was home (matchups were created with team1=Visitor, team2=Home)
# So we want to maximize sum of x[m], or minimize sum of (1 - x[m])

original_match_penalty = []
for m in range(len(matchups)):
    # Original had team2 as home, so we prefer x[m] = 1
    # Penalty for changing: (1 - x[m])
    original_match_penalty.append(1 - x[m])

model.setObjective(gp.quicksum(original_match_penalty), GRB.MINIMIZE)

print(" Objective set: Minimize changes from original home/away assignments")
print("\n" + "="*80)
print("MODEL SUMMARY:")
print("="*80)
print(f"Decision variables: {len(matchups)} (home/away assignment for each matchup)")
print(f"Constraints:")
print(f"  - Home/away balance: {len(teams_list)} (one per team)")
print(f"  - Triple feasibility: {triple_constraints_count} (eliminate infeasible triples)")
print(f"  - Timezone definitions: {sum(len(teams[t]) for t in teams_list)}")
print(f"Objective: Minimize deviations from original schedule")
print("="*80)


 Objective set: Minimize changes from original home/away assignments

MODEL SUMMARY:
Decision variables: 128 (home/away assignment for each matchup)
Constraints:
  - Home/away balance: 16 (one per team)
  - Triple feasibility: 224 (eliminate infeasible triples)
  - Timezone definitions: 256
Objective: Minimize deviations from original schedule


In [15]:
# Solve the model
print("\n" + "="*80)
print("SOLVING MODEL...")
print("="*80)

model.optimize()

print("\n" + "="*80)
print("OPTIMIZATION RESULTS:")
print("="*80)

if model.status == GRB.OPTIMAL:
    print(" OPTIMAL SOLUTION FOUND!")
    print(f"\nObjective value: {model.objVal:.0f} matchups changed from original")
    print(f"Total matchups: {len(matchups)}")
    print(f"Percentage changed: {100 * model.objVal / len(matchups):.1f}%")
    print(f"Percentage unchanged: {100 * (1 - model.objVal / len(matchups)):.1f}%")
    
elif model.status == GRB.INFEASIBLE:
    print(" MODEL IS INFEASIBLE")
    print("Cannot find a schedule that satisfies all constraints")
    print("Consider relaxing some constraints")
    
elif model.status == GRB.TIME_LIMIT:
    print(" TIME LIMIT REACHED")
    print(f"Best solution found: {model.objVal:.0f} matchups changed")
    
else:
    print(f" Optimization ended with status: {model.status}")



SOLVING MODEL...
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.6.0 24G90)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Academic license 2705371 - for non-commercial use only - registered to rc___@columbia.edu
Optimize a model with 1375 rows, 832 columns and 3575 nonzeros

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Academic license 2705371 - for non-commercial use only - registered to rc___@columbia.edu
Optimize a model with 1375 rows, 832 columns and 3575 nonzeros
Model fingerprint: 0xe06ffc05
Variable types: 0 continuous, 832 integer (128 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 3e+00]
  RHS range        [1e+00, 3e+00]

Model fingerprint: 0xe06ffc05
Variable types: 0 continuous, 832 integer (128 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objec

In [14]:
# Computing an Irreducible Inconsistent Subsystem (IIS)
import sys

status = model.status
if status == GRB.UNBOUNDED:
    print('The model cannot be solved because it is unbounded')
    sys.exit(0)
if status == GRB.OPTIMAL:
    print('The optimal objective is %g' % model.objVal)
    sys.exit(0)
if status != GRB.INF_OR_UNBD and status != GRB.INFEASIBLE:
    print('Optimization was stopped with status %d' % status)
    sys.exit(0)

# do IIS
print('The model is infeasible; computing IIS')
removed = []

# Loop until we reduce to a model that can be solved
while True:

    model.computeIIS()
    print('\nThe following constraint cannot be satisfied:')
    for c in model.getConstrs():
        if c.IISConstr:
            print('%s' % c.constrName)
            # Remove a single constraint from the model
            removed.append(str(c.constrName))
            model.remove(c)
            break
    print('')

    model.optimize()
    status = model.status

    if status == GRB.UNBOUNDED:
        print('The model cannot be solved because it is unbounded')
        sys.exit(0)
    if status == GRB.OPTIMAL:
        break
    if status != GRB.INF_OR_UNBD and status != GRB.INFEASIBLE:
        print('Optimization was stopped with status %d' % status)
        sys.exit(0)

print('\nThe following constraints were removed to get a feasible LP:')
print(removed)

The model is infeasible; computing IIS
Academic license 2705371 - for non-commercial use only - registered to rc___@columbia.edu
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.6.0 24G90)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads


Computing Irreducible Inconsistent Subsystem (IIS)...

           Constraints          |            Bounds           |  Runtime
      Min       Max     Guess   |   Min       Max     Guess   |
--------------------------------------------------------------------------
        0      1376         -         0      1408         -           0s
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.6.0 24G90)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads


Computing Irreducible Inconsistent Subsystem (IIS)...

           Constraints          |            Bounds           |  Runtime
      Min       Max     Guess   |   M

In [16]:
# Extract and save optimized schedule
if model.status == GRB.OPTIMAL or model.status == GRB.TIME_LIMIT:
    
    # Create new games dataframe with optimized home/away assignments
    optimized_games = []
    changes = []
    
    for m, matchup in enumerate(matchups):
        team1, team2 = matchup['team1'], matchup['team2']
        date = matchup['date']
        
        # Check if team2 is home (x[m] = 1)
        if x[m].X > 0.5:  # team2 is home
            home_team = team2
            visitor_team = team1
            changed = False  # This was original assignment
        else:  # team1 is home
            home_team = team1
            visitor_team = team2
            changed = True  # This is changed from original
        
        # Get arena (home team's arena from original data)
        arena_info = games_df[games_df['Home'] == home_team].iloc[0]
        arena = arena_info['Arena']
        
        optimized_games.append({
            'Date': date,
            'Visitor': visitor_team,
            'Home': home_team,
            'Arena': arena,
            'LOG': f"{visitor_team} @ {home_team}"
        })
        
        if changed:
            changes.append({
                'Date': date,
                'Original_Home': team2,
                'Original_Visitor': team1,
                'New_Home': home_team,
                'New_Visitor': visitor_team
            })
    
    # Save optimized schedule
    optimized_df = pd.DataFrame(optimized_games)
    optimized_df = optimized_df.sort_values('Date').reset_index(drop=True)
    optimized_df.to_csv('games_optimized.csv', index=False)
    
    print(f"\n Saved optimized schedule to 'games_optimized.csv'")
    print(f"   Total games: {len(optimized_df)}")
    
    if len(changes) > 0:
        changes_df = pd.DataFrame(changes)
        changes_df.to_csv('schedule_changes.csv', index=False)
        print(f"\n Saved {len(changes)} changes to 'schedule_changes.csv'")
        print("\nSample of changes:")
        print(changes_df.head(10).to_string(index=False))
    else:
        print("\n No changes needed - original schedule was already feasible!")



 Saved optimized schedule to 'games_optimized.csv'
   Total games: 128

 Saved 19 changes to 'schedule_changes.csv'

Sample of changes:
             Date         Original_Home      Original_Visitor              New_Home           New_Visitor
Sat, Nov 01, 2025        Boston Celtics Golden State Warriors Golden State Warriors        Boston Celtics
Sat, Nov 01, 2025       New York Knicks    Los Angeles Lakers    Los Angeles Lakers       New York Knicks
Sat, Nov 01, 2025         Brooklyn Nets        Denver Nuggets        Denver Nuggets         Brooklyn Nets
Mon, Nov 03, 2025    Los Angeles Lakers        Boston Celtics        Boston Celtics    Los Angeles Lakers
Mon, Nov 03, 2025 Golden State Warriors        Denver Nuggets        Denver Nuggets Golden State Warriors
Mon, Nov 03, 2025          Phoenix Suns       New York Knicks       New York Knicks          Phoenix Suns
Wed, Nov 05, 2025 Golden State Warriors       Houston Rockets       Houston Rockets Golden State Warriors
Fri, Nov 07, 20

In [17]:
# Verify the optimized schedule has no infeasible triples
if model.status == GRB.OPTIMAL or model.status == GRB.TIME_LIMIT:
    
    print("\n" + "="*80)
    print("VERIFICATION: Checking optimized schedule for infeasible triples")
    print("="*80)
    
    # Recreate chronological schedule with timezones for optimized schedule
    optimized_df = pd.read_csv('games_optimized.csv')
    
    # Arena to timezone mapping
    arena_timezone = {
        'TD Garden': 0, 'Barclays Center': 0, 'Madison Square Garden': 0,
        'Wells Fargo Center': 0, 'Scotiabank Arena': 0, 'State Farm Arena': 0,
        'Kaseya Center': 0, 'Rocket Mortgage FieldHouse': 0,
        'United Center': -1, 'Fiserv Forum': -1, 'American Airlines Center': -1,
        'Toyota Center': -1, 'Ball Arena': -2, 'Footprint Center': -2,
        'Chase Center': -3, 'Crypto.com Arena': -3
    }
    
    # Build team schedules
    all_schedules = []
    for team in teams_list:
        # Home games
        home_games = optimized_df[optimized_df['Home'] == team]
        for _, game in home_games.iterrows():
            all_schedules.append({
                'Team': team,
                'Date': game['Date'],
                'Location': 'Home',
                'Opponent': game['Visitor'],
                'Arena': game['Arena'],
                'Timezone': arena_timezone[game['Arena']]
            })
        
        # Away games
        away_games = optimized_df[optimized_df['Visitor'] == team]
        for _, game in away_games.iterrows():
            all_schedules.append({
                'Team': team,
                'Date': game['Date'],
                'Location': 'Away',
                'Opponent': game['Home'],
                'Arena': game['Arena'],
                'Timezone': arena_timezone[game['Arena']]
            })
    
    optimized_chrono_df = pd.DataFrame(all_schedules)
    optimized_chrono_df['Date'] = pd.to_datetime(optimized_chrono_df['Date'])
    optimized_chrono_df = optimized_chrono_df.sort_values(['Team', 'Date']).reset_index(drop=True)
    optimized_chrono_df['Game_Number'] = optimized_chrono_df.groupby('Team').cumcount() + 1
    optimized_chrono_df = optimized_chrono_df[['Team', 'Game_Number', 'Date', 'Location', 'Opponent', 'Arena', 'Timezone']]
    
    # Check for infeasible triples
    infeasible_count = 0
    for team in teams_list:
        team_games = optimized_chrono_df[optimized_chrono_df['Team'] == team].reset_index(drop=True)
        
        for g in range(len(team_games) - 2):
            tz1 = team_games.loc[g, 'Timezone']
            tz2 = team_games.loc[g+1, 'Timezone']
            tz3 = team_games.loc[g+2, 'Timezone']
            
            total_diff = abs(tz2 - tz1) + abs(tz3 - tz2)
            
            if total_diff >= 4:
                infeasible_count += 1
                print(f" {team}: Games {g+1}-{g+2}-{g+3} have TZ sum = {total_diff}")
    
    if infeasible_count == 0:
        print(" SUCCESS! No infeasible triples found in optimized schedule!")
        print("   All teams have feasible consecutive game timezones")
    else:
        print(f" WARNING: {infeasible_count} infeasible triples still exist")
    
    # Save optimized chronological schedule
    optimized_chrono_df.to_csv('team_chronological_schedule_optimized.csv', index=False)
    print(f"\n Saved optimized chronological schedule to 'team_chronological_schedule_optimized.csv'")
    
    print("\n" + "="*80)



VERIFICATION: Checking optimized schedule for infeasible triples
 Boston Celtics: Games 1-2-3 have TZ sum = 5
 Brooklyn Nets: Games 7-8-9 have TZ sum = 4
 Los Angeles Lakers: Games 1-2-3 have TZ sum = 6
 Los Angeles Lakers: Games 2-3-4 have TZ sum = 5
 Los Angeles Lakers: Games 6-7-8 have TZ sum = 4
 New York Knicks: Games 7-8-9 have TZ sum = 4
 Phoenix Suns: Games 2-3-4 have TZ sum = 4
 Phoenix Suns: Games 7-8-9 have TZ sum = 4

 Saved optimized chronological schedule to 'team_chronological_schedule_optimized.csv'

